# Transformers
---
## Objectives
1. Motivation for Transformers
2. Define Transformers
3. Define Self-Attention
    1. Self-Attention with vectors
    2. Self-Attention with matrices
4. Define Multi-Head Attention
5. Define Encoder-Decoder Attention layer
6. Final Linear & Softmax Layers
7. Loss Function

## Motivation
A Transformer is a type of neural network architecture developed to solve the problem of **sequence transduction** or **neural machine translation**: any task that transforms an input sequence to an output sequence.

* speech recognition
* text-to-speech transformation
* language translation

Useful when words from a sentence refer to phrases from a sentence prior:

"Tomato basil is my favorite starter because it is surprisingly filling." 

*It* referring to *tomato basil*

### Other Use Cases
* [Biological structure and function emerge from scaling unsupervised learning to 250 million protein sequences](https://www.biorxiv.org/content/10.1101/622803v2)
* [Generating Long Sequences with Sparse Transformers](https://arxiv.org/abs/1904.10509)
* [Selfie: Self-supervised Pretraining for Image Embedding](https://arxiv.org/abs/1906.02940)
* [Behavior Sequence Transformer for E-commerce
Recommendation in Alibaba](https://arxiv.org/pdf/1905.06874.pdf)


## But first, what's Seq2Seq?
Sequence-2-Sequence (Seq2Seq) models are made of two components: an **Encoder** and a **Decoder**.

"The **Encoder** turns each item into a corresponding hidden vector containing the *item and its context*. The **Decoder** reverses the process, turning the vector into an output item, using the previous output as the input context." --[seq2seq model in Machine Learning](https://www.geeksforgeeks.org/seq2seq-model-in-machine-learning/)

This *context* of each input of the sequence in question is a vector.

<img src="images/seq2seq.png"/>

### Methods previous have issues...

Seq2Seq models need to figure out dependencies like when *soup* refers to *tomato basil* in our example above. We've seen **Recurrent Neural Networks** (RNNs) and **Convolution Neural Networks** (CNNs) so far. RNNs become very inefficient when there is a large enough gap between relevant information. The longer the chain of info, the more likely important info will get lost in that chain. **Long-Short Term Memory** (LSTM), a type of RNN, tried to solve this existing problem of learning long-term dependencies by remembering relevant info and forgetting other info deemed less important.

<img src="images/unrolled_rnn.png" width="75%" height="75%"/>

<img src="images/rnn_decode.gif"/>

* RNNs updates its hidden state based on its inputs and previous inputs it has seen. The hidden state from the final output of the encoder makes it challenging to convey all of the input sequences *context*. This final hidden state or *context* vector turns out to be a bottleneck!

LSTMs make small modifications to the info with multiplications and additions. Recall that the info flows through cell states so LSTMs can select what they *remember* and *forget*.

* The probability of keeping the context from one piece of info that is far from another piece of info decreases exponentially with the distance between inputs of the sequence.

And lastly, BOTH RNNs and LSTMs are hard to parallelize... think you must go *step-by-step* i.e. **sequential = SLOWWWWWW**

### To sum up the problems of RNNs & LSTMs in 3 points:
1. Sequential computation inhibits parallelization
2. No explicit modeling of long and short range dependencies
3. "Distance" between positions is linear


## Solution: Attention by [Bahdanau et al., 2014](https://arxiv.org/abs/1409.0473) & [Luong et al., 2015](https://arxiv.org/abs/1508.04025)
A technique for paying attention to specifics of the information provided; like listening more carefully to the segment I'm writing down during an audio recording or paying attention to certain objects in a room while describing your surroundings.

<img src="images/room.jpeg"/>

**Attention** models differ from the previous Seq2Seq models in two ways:

1. Instead of encoding the ENTIRE SENTENCE in a hidden state, each word from a sentence has a corresponding encoded hidden state that is passed all the way to the decoding stage. There may be relevant info in each word of the sentence (there usually is) so decoding needs to take into account every word of the input.

<img src="images/attention.gif"/>

2. The decoder does one extra step before producing an output: it processes specific parts of the input that are relevant at the current step by looking at each encoded hidden state it received, gives each hidden state a score, then multiplies each hidden state by its *softmaxed score* to amplify hidden states with higher scores.

<img src="images/attention-gif.gif"/>

Attention is essentially a method for amplifying signal from the relevant parts of the input sequence.

Have we solved the initial problems of RNNs and LSTMs? Nope. We have yet to parallelize computations! 

## CNNs to the rescue!

* Parallelize each layer: each input can be processed/encoded at the same time; doesn't depend on the previous word to be translated
* Distance between positions is logarithmic of $N$ where $N$ is the size of the height of the tree generated from the output to the input below (however many hidden layers)

<img src="images/wavenet.gif"/>

CNNs combined with Attention allow us to parallelize computation with logarithmic distances b/t inputs, while also figuring out the problem of dependencies when translating sentences! And yes, there's a name for CNN-Attention combos: **TRANSFORMERS!!!**; first introduced in [Attention is All You Need](https://arxiv.org/abs/1706.03762).

---
Breakout
---
1. List several previous methods for dealing with sequential data.
2. What are the three problems with these methods?
3. How does **Attention** solve two of the three problems?
4. How do **CNNs** solve the last of the three problems?

---
Breakout Solution
---
1. RNNs (Recurrent Neural Network) and LSTMs (Long-Short Term Memory).
2. (1) There is no modeling of dependencies amongst pieces of sequential data or these methods cannot model dependencies *well*, (2) distance between inputs of a sequence are in linear computation time, (3) computations cannot be parallelized.
3. Attention models dependencies amongst pieces of sequential data by amplifying signal around more important inputs of the sequence, and reduces computation to logarithmic time.
4. CNNs allow for parallelized computation.

# Transformers: [The Illustrated Transformer](http://jalammar.github.io/illustrated-transformer/)
---
Transformers are **Attention** added to CNNs to boost training speeds.

We will follow the link above for steps through visualizing transformers at a high-level based off [Attention is All You Need](https://arxiv.org/abs/1706.03762) use case of applying a transformer to translate French to English.

*None of the images are my own-they are from the blog [The Illustrated Transformer](http://jalammar.github.io/illustrated-transformer/) by Jay Alammar.*

Transformers are composed of two major components each with sub-components:
1. **Encoder**

    1. $N$ encoders comprised of a self-attention layer & feed forward neural network layer
2. **Decoder**
    
    1. $N$ decoders comprised of a self-attention layer, encoder-decoder attention layer, and feed-forward neural network layer

## Black Box

<img src="images/transformer-1.png"/>

## Encoding & Decoding Components

<img src="images/transformer-2.png"/>

### Stacks of Encoders & Decoders
Encoding & decoding components are stacks of encoders and decoders (the same amount each), respectively.
<img src="images/transformer-3.png"/>

## Encoders
All identical but do not share weights, broken down into two sub-layers. The first layer, the *self-attention layer*, helps the encoder look at other words in the input sentence as it encodes the current word. This outputs to a feed-forward neural network.

<img src="images/encoder.png"/>

## Decoders
The decoder has both the self-attention and feed forward neural network with an attention layer between. The encoder-decoder attention layer between helps the decoder focus on the relevant parts of the input sentence.

<img src="images/decoder.png"/>

## Attention in Transformers
#### 3 Ways of Attention:
1. Encoder Self-Attention
2. Decoder Self-Attention
3. Encoder-Decoder Attention

<img src="images/3-attention.png"/>

## Self-Attention
---
Now we'll follow the vectors/tensors through the encoder components to better understand **self-attention**.

Start by turning each input word into a vector using an *embedding algorithm*. 

* An easy to follow example of obtaining a word embedding by training a neural network, and then obtaining the hidden layer weights is [Creating Word Embeddings: Coding the Word2Vec Algorithm in Python using Deep Learning](https://towardsdatascience.com/creating-word-embeddings-coding-the-word2vec-algorithm-in-python-using-deep-learning-b337d0ba17a8).
* Similarly the article recommends using embedding weights from [GloVe: Global Vectors for Word Representation](https://nlp.stanford.edu/projects/glove/).

<img src="images/embeddings.png"/>

This embedding only happens in the bottom-most encoder, and in *Attention is All You Need*, they create a list of vectors each of size 512. The size of this list is a hyperparameter than can be set to the longest sentence in our training dataset.

### How do transformers account for sequence order?

Each input embedding gets another vector. These vectors can follow a specific pattern that the model learns, helping it to determine the position of each word (distance b/t inputs in a sequence) or they can be hard coded as *Attention is All You Need* observes with the following equations:

<img src="images/positional-encoding-formula.png"/>

<img src="images/transformer_positional_encoding_example.png"/>

Note that each word in each position flows through its own path in the encoder. There are dependencies between these paths in the self-attention layer, however the feed-forward layer does not have those dependencies therefore the various paths can be executed in parallel while flowing through the feed-forward layer.

<img src="images/encoder_with_tensors_2.png"/>

## *Warning: Math Heavy*
### Calculating **self-attention** using vectors:
Entails 6 steps of MATH. This process is first introduced with vectors but is actually done on the matrix level for faster computation.

$$Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}})V$$

where
* $Q =$ query vector (what model is trying to learn context for)
* $K =$ key vector (determines attention weights)
* $V =$ value vector (get multiplied by attention weights)

Think of this as mapping your current query (current input's hidden state) to the most similar other hidden states (keys) in the sequence. Softmax the results (normalize) and multiply with the keys (also the values?) again, to yield how much attention you should give for each hidden state.

#### First Step
1. Create 3 vectors from each of the encoder's input vectors (embedding + positional encoding of each word) by multiplying the embedding by three weight matrices that we obtain in the training process. Resulting in a **Query vector**, a **Key vector**, and a **Value vector**.
    1. Each of these vectors is a smaller dimension, 64, than the original 512 in *Attention is All You Need*. This isn't necessary but is  chosen to make computation of multihead attention constant.
    
We are multiplying $x1$ by the $W^Q$ weight matrix to produce $q1$, the **query vector** associated with that word.

Similarly multiply $x1$ by the $W^K$ weight matrix to produce $k1$, the **key vector**.

Lastly, multiply $x1$ by the $W^V$ weight matrix to produce $v1$, the **value vector**.
<img src="images/transformer_self_attention_vectors.png"/>

"The key/value/query concepts come from retrieval systems. For example, when you type a query to search for some video on Youtube, the search engine will map your **query** against a set of **keys** (video title, description, etc.) associated with candidate videos in the database, then present you the best matched videos (**values**).

The attention operation turns out can be thought of as a retrieval process as well, so the key/value/query concepts also apply here."  --[Source](https://stats.stackexchange.com/questions/421935/what-exactly-are-keys-queries-and-values-in-attention-mechanisms)

All three transformation weight matrices, $W^Q$, $W^K$, and $W^V$ are learned in a neural network! By multiplying our input embedded vector (one word) by the weight matrices, we get:
1. Increased possibility for each input vector to bring attention to other vectors in the input sequence.
2. Possibly a better latent representation of the input vector.
3. We convert the embedding vector into a different dimension.

#### Second Step
2. Calculate a **score** for each word in the input sequence against the word in question. The score determines how much focus to place on other parts of the input sequence as we encode a word at a certain position.

For example, the following image is calculating the **score** for the word "Thinking" against itself, and against "Machines".

Take the dot product of the **query vector** with the **key vector** of the respective word we're scoring.

<img src="images/transformer_self_attention_score.png"/>

#### Third & Fourth Step
3. Stabilize the gradient by dividing the **scores** by the square root of the dimension of the **key vectors**. In the example we're following along with, that is $\sqrt{64} = 8$.

4. Then pass the result through a softmax operation to normalize the scores so that they are all positive and sum to 1.

"This softmax score determines how much each word will be expressed at this position. Clearly the word at this position will have the highest softmax score, but sometimes it’s useful to attend to another word that is relevant to the current word." --[Blog we're following along with](http://jalammar.github.io/illustrated-transformer/)

<img src="images/self-attention_softmax.png"/>

#### Fifth & Sixth Step
5. Multiply each **value vector** by the softmax score to keep intact the values of the words we want to focus on, and drown-out irrelevant words.

6. Sum the weighted value vectors produced in step 5. This is the output of the self-attention layer **AT THIS POSITION** i.e. **FOR THE FIRST WORD**.

<img src="images/self-attention-output.png"/>

The resulting vector is what is sent to the [feed-forward neural network](https://en.wikipedia.org/wiki/Feedforward_neural_network). This happens in matrix form, not word-by-word, for WAY faster processing!

---
Breakout
---
1. What is a Transformer in the most general sense?
2. What are the two main components of a transformer and what are their subcomponents?
3. What does the third layer of each decoder contribute?
4. What is a crucial step prior to feeding text to a transformer in Natural Language Processing? (This step is done in the first encoder in a transformer)
5. What are the three matrices trained in the self-attention layer of an encoder and what vectors do they calculate when multiplied with one word's embedding vector?

---
Breakout Solution
---
1. A Transformer is a CNN combined with self-attention.
2. Encoder component comprised of $N$ encoder subcomponents, each comprised of a self-attention layer and feed-forward neural network layer. Decoder component comprised of $N$ decoder subcomponents, each comprised of a self-attention layer, a encoder-decoder attention layer, and a feed-forward neural network layer.
3. The encoder-decoder attention layer between helps the decoder focus on the relevant parts of the input sentence.
4. Word embedding: transforming text to vectors, and combining with a positional encoding to create an embedding with time signal.
5. $W^Q$, $W^K$, and $W^V$ and they calculate the **query**, **key**, and **value** vectors, respectively.

### Calculating **self-attention** using matrices:

#### First Step
1. Calculate the **Query**, **Key**, **Value** matrices. Pack each word embedding into a matrix $X$ (where each row in $X$ corresponds to a word in the input sequence) and multiply by the weight matrices obtained through training, $W^Q$, $W^K$, and $W^V$.

<img src="images/self-attention-matrix-calculation.png"/>

#### Steps 2-6

Condensed down to one formula to produce the outputs of the self-attention layer:

<img src="images/self-attention-matrix-calculation-2.png"/>

---
Breakout/Motivation
---
So far, we worked with one vector, and then just one matrix.

The resulting softmaxed matrix multiplied by the **value** matrix in the self-attention layer results in what we can think of as a *focus*/*attention* matrix, for each input relative to the other inputs in the sequence. The final output of the self-attention layer, the matrix $Z$, will contain a little bit of every other encoding relative to the current input, but could have some issues. Discuss what these issues may be.

---
Breakout/Motivation Solution
---
Intuitively, self-attention compensates for one *type of dependency* of different parts of a sequence relative to each other. What if this one dependency was only in the short-term? Would we need to calculate another for long-term dependencies?

Another way to think of this is in terms of speaking a language. Do you speak in a monotone voice? How do you convey emotion through speaking? Using *nuance*. We can think of nuance as adding color to your language. Nuance is the subtle difference in speaking in blue (maybe blue is calm??) vs. light-blue vs. deep-blue, or vs. speaking in yellow (is yellow yelling??)! Nuance is adding emphasis or de-emphasizing certain parts of a sentence to convey deeper meaning. Can one such subspace/matrix convey ALL POSSIBLE NUANCES of that word relative to the others in a sentence? Probably not...

<img src="images/speaking.png"/>

Each word in a language can have multiple relationships and nuances. One attention *head*, or computation, can only code one such relationship/nuance. We would like our Transformer to learn *many* word relationships and nuances in our language to be able to translate their meaning accurately.

**Advanced**: How can we think of these dependencies in high dimensional spaces?

**Multiple attention heads** allows for attending to parts of the sequence differently (i.e. longer-term dependencies versus shorter-term dependencies).

## Multi-Head Attention

To allow our transformer to learn multiple dependencies and nuances of an input relative to the other inputs of a sequence, we introduce multiple attention heads. This adds two improvements:

1. Expands the model's ability to focus on different positions. The softmax computation will result in our output matrix being dominated by each word relative to itself in its corresponding row.
2. It gives the attention layer multiple "representation subspaces" or what we can think of as dependencies/nuances. For instance, one head might capture the ‘gender-ness’ (male, female, netral) of a noun while another might capture the ‘cardinality’ (singular vs plural) of a noun. This might be important during translation because, in many languages, the verb that needs to be used depends on these factors.

The Attention layer repeats its computations multiple times in parallel where each is called an *attention head*. This is best visualized:

<img src="images/multi-head.png"/>

"The Attention module splits its Query, Key, and Value parameters N-ways and passes each split independently through a separate Head. All of these similar Attention calculations are then combined together to produce a final Attention score." --[Transformers Explained Visually (Part 3): Multi-head Attention, deep dive](https://towardsdatascience.com/transformers-explained-visually-part-3-multi-head-attention-deep-dive-1c1ff1024853)

As the image below illustrates, each attention head will have its own $Q$, $K$, and $V$ weight matrices. *Attention is All You Need* calculates self-attention 8 different times, using 8 attention heads. Therefore we have 8 different $Z$ output matrices.

Condensing these 8 $Z$ matrices to one with one additional weights matrix $W^O$ to pass along to the feed-forward neural network.

To wrap up multi-head attention, Jay condenses the 8 attention heads to one visual:

<img src="images/transformer_multi-headed_self-attention-recap.png"/>


## Layer Normalization

In order to reduce training time, each layer's outputs are normalized as introduced in [Layer Normalization](https://arxiv.org/abs/1607.06450). The following image is a simplified transformer with 2 stacked encoders and decoders.

<img src="images/transformer_resideual_layer_norm_3.png"/>

## Encoder-Decoder Attention
"In the decoder, the self-attention layer is only allowed to attend to earlier positions in the output sequence. This is done by masking future positions (setting them to -inf) before the softmax step in the self-attention calculation.

The “Encoder-Decoder Attention” layer works just like multiheaded self-attention, except it creates its Queries matrix from the layer below it, and takes the Keys and Values matrix from the output of the encoder stack." --[The Illustrated Transformer](http://jalammar.github.io/illustrated-transformer/)

## Wrap it up: Final Linear and Softmax Layer

"The following steps repeat the process until a special symbol is reached indicating the transformer decoder has completed its output. The output of each step is fed to the bottom decoder in the next time step, and the decoders bubble up their decoding results just like the encoders did. And just like we did with the encoder inputs, we embed and add positional encoding to those decoder inputs to indicate the position of each word." --[The Illustrated Transformer](http://jalammar.github.io/illustrated-transformer/)

<img src="images/transformer_decoding_2.gif"/>

The last Linear layer and Softmax layer translate the decoder stack output vector of floats into a word.

**The Linear Layer**: "...a simple fully connected neural network that projects the vector produced by the stack of decoders, into a much, much larger vector called a logits vector.

Let’s assume that our model knows 10,000 unique English words (our model’s “output vocabulary”) that it’s learned from its training dataset. This would make the logits vector 10,000 cells wide – each cell corresponding to the score of a unique word. That is how we interpret the output of the model followed by the Linear layer."

**The Softmax Layer**: turns the logits scores into probabilities which the transformer chooses from to select the highest probability and outputs the word that it corresponds to.

<img src="images/transformer_decoder_output_softmax.png"/>

---
Breakout
---
1. With multi-head attention we end up with multiple $Z$ matrices corresponding to each attention head. How does the multi-head attention layer output one single $Z$ matrix?
2. What additional layer is added after each encoder & decoder subcomponent layers and why?
3. Describe how the encoder-decoder attention layer in decoders is different than self-attention layers.
4. What does the final linear layer output? What about the final softmax layer?

---
Breakout Solutions
---
1. Concatenating each head's resulting $Z$ matrix and multiplying by a final weight matrix $W^O$.
2. Normalization to speed up training time and stabilize the gradient.
3. The encoder-decoder layer uses the Queries matrix from the self-attention layer previous (in the decoder), and the Keys and Values Matrices from the encoder.
4. The linear layer outputs a logits vector the length of the vocabulary the transformer has been trained on. The softmax layer transforms the logits vector into a log probability vector of the same length.

## Loss Function & Training
We would like the output of our transformer to be a probability distribution indicating a word for each input of a sequence. To compare two probability distributions Jay points us to [cross-entropy](https://colah.github.io/posts/2015-09-Visual-Information/).

"For example – input: “je suis étudiant” and expected output: “i am a student”. What this really means, is that we want our model to successively output probability distributions where:

* Each probability distribution is represented by a vector of width `vocab_size` (6 in our toy example, but more realistically a number like 30,000 or 50,000)
* The first probability distribution has the highest probability at the cell associated with the word “i”
* The second probability distribution has the highest probability at the cell associated with the word “am”
* And so on, until the fifth output distribution indicates `<end of sentence>` symbol, which also has a cell associated with it from the 10,000 element vocabulary."

<img src="images/transformer.png"/>

---
Review
---
1. Name the three places attention is used in transformers.
2. What are the three input parameters (matrices) for attention?
3. Why would we use multi-head attention as opposed to just one layer of attention?
4. How does an encoder's subcomponents differ from a decoder's subcomponents?
5. How do we choose how many encoders and decoders the encoder and decoder components of a transformer has?
6. Why do we need the final linear and softmax layers after the encoder and decoder components of a transformer?
7. How does a transformer know the order of the sequence it is encoding/decoding?

---
Assignment
---
Build a Transformer from scratch. 

Solutions can reference [CVxTz's Github](https://github.com/CVxTz) for music genre classification: https://github.com/CVxTz/music_genre_classification/blob/master/code/transformer.py

# References/Resources
---
* [Visualizing Machine Neural Translation](https://jalammar.github.io/visualizing-neural-machine-translation-mechanics-of-seq2seq-models-with-attention/)
* [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473)
* [Effective Approaches to Attention-based Neural Machine Translation](https://arxiv.org/abs/1508.04025)
* [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
* [Illustrated Transformers](http://jalammar.github.io/illustrated-transformer/)

* [Creating Word Embeddings: Coding the Word2Vec Algorithm in Python using Deep Learning](https://towardsdatascience.com/creating-word-embeddings-coding-the-word2vec-algorithm-in-python-using-deep-learning-b337d0ba17a8)
* [GloVe: Global Vectors for Word Representation](https://nlp.stanford.edu/projects/glove/)
    For word embedding weights
* [Generating Long Sequences with Sparse Transformers](https://arxiv.org/abs/1904.10509)
* [Layer Normalization](https://arxiv.org/abs/1607.06450)
* [Visual Information Theory](https://colah.github.io/posts/2015-09-Visual-Information/)